In [1]:
from agents import Agent, Runner, AsyncOpenAI, set_default_openai_client, set_default_openai_api
from agents.tool import function_tool
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()

AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_MODEL = os.getenv("AZURE_OPENAI_MODEL")

OPEN_WEATHER_API_KEY = os.getenv("OPENWEATHER_API_KEY")


external_client = AsyncOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    base_url=f"{AZURE_OPENAI_ENDPOINT}/openai/deployments/{AZURE_OPENAI_DEPLOYMENT_NAME}",
    default_headers={"api-key": AZURE_OPENAI_API_KEY},
    default_query={"api-version": AZURE_OPENAI_API_VERSION}
)

set_default_openai_client(external_client, use_for_tracing=False)
set_default_openai_api("chat_completions")

In [3]:
from axios import Axios

httpClient = Axios()

In [4]:
from pydantic import BaseModel

class GeoLocation:
  def __init__(self, name, lat, lon):
    self.name = name
    self.lat = lat
    self.lon = lon

  @classmethod
  def from_json(cls, json_data: dict):
    return cls(
      name=json_data.get('name'),
      lat=json_data.get('lat'),
      lon=json_data.get('lon')
    )
  
  def to_dict(self):
    return {
      "name": self.name,
      "lat": self.lat,
      "lon": self.lon
    }

  def __repr__(self):
    return f"GeoLocation(name='{self.name}', lat='{self.lat}', lon='{self.lon}')"
  

class WeatherInfo():
  def __init__(self, main, temp_min, temp_max):
    self.main = main
    self.temp_min = temp_min
    self.temp_max = temp_max

  @classmethod
  def from_json(cls, json_data: dict):
    weather = json_data.get('weather')
    main = json_data.get('main')

    return cls(
      main=weather[0].get('main'),
      temp_min=main.get('temp_min'),
      temp_max=main.get('temp_max')
    )
  
  def to_dict(self):
    return {
      "main": self.main,
      "temp_min": self.temp_min,
      "temp_max": self.temp_max
    }
  
  def __repr__(self):
    return f"WeatherInfo(main='{self.main}', min='{self.temp_min}', max='{self.temp_max}')"

In [5]:
@function_tool
async def get_geo(city: str):
  """
  Get geolocation details of a city.
  Parameters:
    - city (str): The name of the city to get location info for.
  Returns:
    - {"lat": int, "lon": int}: Basic location info or coordinates.
  """
  url = f"http://api.openweathermap.org/geo/1.0/direct?q={city}&appid={OPEN_WEATHER_API_KEY}"
  response = await httpClient.aget(url)
  geo = GeoLocation.from_json(response.data[0])
  return geo.to_dict()
  


@function_tool
async def get_forecast(lat, lon):
  """
  Get weather details of a city.
  Parameters:
    - lat (number): Latitude of a city.
    - lon (number): Longitude of a city.
  Returns:
    - {"main": string, "temp_min": number, "temp_max": number}: Weather info contains main, min temperature and max temperature.
  """
  url=f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={OPEN_WEATHER_API_KEY}&units=metric"
  response = await httpClient.aget(url)
  info = WeatherInfo.from_json(response.data)
  return info.to_dict()

In [6]:
"""
geolocation = await get_geo("Petaling Jaya")
print(geolocation)
weatherinfo = await get_forecast(geolocation)
print(weatherinfo)
"""


'\ngeolocation = await get_geo("Petaling Jaya")\nprint(geolocation)\nweatherinfo = await get_forecast(geolocation)\nprint(weatherinfo)\n'

In [7]:
instructions = """
You extract the city name from the text in any given language. \
You only use the given tools. \
You get the result using get_geo tool using the city name. \
You get the weather info using get_forecast using the lat and lon of get_tool result. \
You display the lat and lon of the get_geo. \
You display the weahter info of the get_forecast. \
You only speak Mandarin and translate the city name.
"""

chinese_agent = Agent(
  name = "Chinese Agent",
  instructions = instructions,
  tools = [get_geo, get_forecast]
)

In [8]:
import asyncio

In [9]:
async def run(city: str):
  result = await Runner.run(chinese_agent, city)
  print(result.final_output)

In [10]:
await run('What is the weather for Petaling Jaya today?')

城市的名字是八打灵再也。 

八打灵再也的地理坐标为：  
纬度：3.0991752  
经度：101.6450232  

今天的天气为：多云  
最低气温：32.52°C  
最高气温：34.81°C  
